### NLP Setup

In [ ]:
import pickle
import sys
import spacy

# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

### Import the Template

In [ ]:
from app.templates.meat_sale.nl_template import nl_template

obs = nl_template['obligations']

print('OBLIGATIONS')
for ob in obs:
    print(f'{ob}: {obs[ob]}\n')

In [ ]:
from app.templates.meat_sale.contract_template import get_template

contract_template = get_template()

print(contract_template.to_sym())

In [ ]:
from app.src.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

In [ ]:
from app.classes.spec.symboleo_spec import PAtom
from app.src.graph.graph_builder import GraphBuilder
from app.src.graph.graph_visualizer import GraphVisualizer
from app.src.dynamic_constructor import DynamicObjectConstructor

graph_builder = GraphBuilder()
graph_visualizer = GraphVisualizer()

digraph = graph_builder.build(PAtom)

gv = graph_visualizer.create_viz(digraph.nodes)
gv.show('nx.html')

dynamic_constructor = DynamicObjectConstructor(digraph)


### DELIVERY_TIMEFRAME

In [ ]:
from app.src.rules.shared.predicate_processor import PredicateProcessor
from app.src.rules.meat_sale.delivery_timeframe.dtf_config import get_dtf_pred_config
from app.src.rules.shared.configs import PredicateExtractorConfig, PredicateProcessorConfig
from app.src.dynamic_constructor import DynamicObjectConstructor
from app.src.rules.shared.predicate_extractor import PredicateExtractor
from app.src.norm_proposition_updater import NormPropositionUpdater

pred_proc_config = PredicateProcessorConfig(
    norm_type = 'obligations',
    norm_id = 'delivery',
    norm_component = 'consequent',
)
dtf_predex_config = get_dtf_pred_config(nlp)
dynamic_constructor = DynamicObjectConstructor(digraph)
norm_updater = NormPropositionUpdater()
pred_config = PredicateExtractorConfig
predicate_extractor = PredicateExtractor(nlp, dtf_predex_config, dynamic_constructor)
dtf_processor = PredicateProcessor(pred_proc_config, predicate_extractor, norm_updater)


### DELIVERY_LOCATION

In [ ]:
# Want to shift much of this to a setup function or a config...
from app.src.rules.shared.domain_prop_processor import DomainPropProcessor
from app.src.rules.meat_sale.delivery_location.location_extractor import LocationExtractor
from app.src.rules.meat_sale.delivery_location.address_scorer import AddressScorer
from app.src.rules.meat_sale.delivery_location.noun_chunk_scorer import NounChunkScorer
from app.src.rules.meat_sale.delivery_location.role_scorer import RoleScorer

from app.src.rules.meat_sale.delivery_location.address_extractor import AddressExtractor
from app.src.rules.meat_sale.delivery_location.role_score_builder import RoleScoreBuilder
from app.src.rules.meat_sale.delivery_location.coref_getter import CorefGetter
from app.src.rules.meat_sale.delivery_location.matchers import get_matcher
from app.src.rules.shared.configs import DomainPropProcessorConfig

address_extractor = AddressExtractor()
coref_getter = CorefGetter(nlp)

init_scores = {
    'seller': 0.5,
    'buyer': 0.6
}
role_score_builder = RoleScoreBuilder(nlp, init_scores, coref_getter)

scorers = [
    AddressScorer(nlp, address_extractor),
    NounChunkScorer(nlp),
    RoleScorer(nlp, role_score_builder)
]

matcher = get_matcher(nlp)
location_extractor = LocationExtractor(nlp, scorers, matcher)

dl_config = DomainPropProcessorConfig(
    'events',
    'delivered',
    'delDueDate',
    'str'
)

dl_processor = DomainPropProcessor(dl_config, location_extractor)

### Contract Updater

In [ ]:
from app.src.contract_updater import ContractUpdater, ContractUpdateRequest
from app.src.processor_lookup import ProcessorLookup

processor_dict = {
    'DELIVERY_TIMEFRAME': dtf_processor,
    'DELIVERY_LOCATION': dl_processor
}
processor_lookup = ProcessorLookup(processor_dict)

contract_updater = ContractUpdater(processor_lookup)


In [ ]:
contract = contract_template

In [ ]:
user_key = 'DELIVERY_TIMEFRAME'
user_value = 'within 2 weeks'

# user_key = 'DELIVERY_LOCATION'
# user_value = 'at the buyer\'s warehouse'

doc = nlp(user_value)

### Run

In [ ]:
req = ContractUpdateRequest(
    contract, 
    user_key,
    user_value,
    doc
)
contract = contract_updater.update(req)

print(contract.to_sym())